# Extract the SMAP soil moisutre

## Calculate the row and column number for each site

In [1]:
import os
import pandas as pd
import numpy as np
from osgeo import gdal
from osgeo import ogr
from TimeseriesExtractor import PointGeometry

# The projection bounds and grid size of EASE 2.0 used in SMAP products, https://nsidc.org/ease/ease-grid-projection-gt
# Bounding Rectangle: N: 85.044 S: -85.044 E: 180.0 W: -180.0
EASE_ulx = -17367530.45
EASE_uly = 7314540.83
pixelWidth = 36032.22 #36032.22
pixelHeight = -36032.22 #36032.22
SOURCE_EPSG=4326 # WGS 84
TARGET_EPSG=6933 # EASE 2.0 EPSG
HOME_DIR = r"E:\Zoho WorkDrive (YICODE)\My Folders\TimeSeriesRetrieval\Extension"
SM_SITES = os.path.join(HOME_DIR, "site_info.csv") # the site informaiton extracted by Preprocessing_ISMN_Raw_Data.ipynb

In [2]:
sites = pd.read_csv(SM_SITES, float_precision="high")
if 'EASE_row' not in sites: # EASE row and column are extracted if they are not in the site_info.csv 
    p_geo = PointGeometry(SOURCE_EPSG,TARGET_EPSG)
    column_offset=[]
    row_offset=[]
    for site_idx, site in sites.iterrows():
        loc = p_geo.re_project(site.lon, site.lat)
        column_offset.append(int(np.floor((loc[0] - EASE_ulx) / pixelWidth)))
        row_offset.append(int(np.floor((loc[1] - EASE_uly) / pixelHeight)))
    sites['EASE_row']=row_offset # inlcude the row and column number for each site
    sites['EASE_column']=column_offset
    sites.to_csv(SM_SITES,index=False)

## Extract the soil moisutre over all sites

In [3]:
requiredCR_df=sites.groupby(['EASE_row','EASE_column']).size().reset_index().rename(columns={0:'count'})

In [4]:
import h5py
import glob
import numpy as np
import re
data_dir = r'F:\SMAP\36km' # dir of the raw SMAP data
output_dir = r'E:\Zoho WorkDrive (YICODE)\My Folders\TimeSeriesRetrieval\Extension\SMAP'
column_name=['r%sc%s' % (row['EASE_row'], row['EASE_column']) for index, row in requiredCR_df.iterrows()]

df_timeframe = pd.date_range(start="2015-12-01", end="2019-12-31", freq='d').rename('time').to_frame().reset_index(drop=True)
df_timeframe.set_index('time',inplace=True)
extracted_sm=[]
file_dates=[]
h5_list = glob.glob(data_dir + '\*.h5')
for h5_file in h5_list:
    date_ds = re.search(r'\d{8}', h5_file).group()
    if np.sum(df_timeframe.index==date_ds):
        ds = h5py.File(h5_file, 'r')
        #sm_am = ds['Soil_Moisture_Retrieval_Data_AM']['soil_moisture'][:, :]
        sm_am = ds['Soil_Moisture_Retrieval_Data_AM']['soil_moisture'][:, :]
        extracted_sm.append(sm_am[requiredCR_df['EASE_row'],requiredCR_df['EASE_column']])
        file_dates.append(date_ds)
extracted_sm=np.asarray(extracted_sm)
extracted_sm[extracted_sm==-9999]=np.nan
obv_var=pd.DataFrame(extracted_sm,columns=column_name)
obv_var['time']=pd.to_datetime(file_dates)
obv_var.set_index('time',inplace=True)
obv_var=pd.concat([df_timeframe,obv_var],axis=1)
obv_var.to_csv(os.path.join(output_dir,'SMAP.csv'))

## Extract the S, C and bulk density of each site from SMAP products 

In [6]:
import h5py
import glob
import numpy as np
import re
import os
data_dir = r'F:\SMAP\36km' # dir of the raw SMAP data
output_dir = r'E:\Zoho WorkDrive (YICODE)\My Folders\TimeSeriesRetrieval\Extension\SMAP'

if not os.path.exists(os.path.join(output_dir,'bulk_density.csv')):
    column_name=['r%sc%s' % (row['EASE_row'], row['EASE_column']) for index, row in requiredCR_df.iterrows()]

    df_timeframe = pd.date_range(start="2015-12-01", end="2019-12-31", freq='d').rename('time').to_frame().reset_index(drop=True)
    df_timeframe.set_index('time',inplace=True)
    extracted_bd=[]
    file_dates=[]
    h5_list = glob.glob(data_dir + '\*.h5')
    for h5_file in h5_list:
        date_ds = re.search(r'\d{8}', h5_file).group()
        if np.sum(df_timeframe.index==date_ds):
            ds = h5py.File(h5_file, 'r')
            #sm_am = ds['Soil_Moisture_Retrieval_Data_AM']['soil_moisture'][:, :]
            bd = ds['Soil_Moisture_Retrieval_Data_AM']['bulk_density'][:, :]
            extracted_bd.append(bd[requiredCR_df['EASE_row'],requiredCR_df['EASE_column']])
            file_dates.append(date_ds)
    extracted_bd=np.asarray(extracted_bd)
    extracted_bd[extracted_bd==-9999]=np.nan
    obv_var=pd.DataFrame(extracted_bd,columns=column_name)
    obv_var['time']=pd.to_datetime(file_dates)
    obv_var.set_index('time',inplace=True)
    obv_var=pd.concat([df_timeframe,obv_var],axis=1)
    obv_var=obv_var.mean().to_frame().T
    obv_var.to_csv(os.path.join(output_dir,'bulk_density.csv'),index=False)
else:
    obv_var = pd.read_csv(os.path.join(output_dir,'bulk_density.csv'))
        

In [7]:
sites = pd.read_csv(SM_SITES, float_precision="high")
if 'roh_b' not in sites:
    roh_b=[]
    for site_idx, site in sites.iterrows():
        roh_b.append(obv_var['r%sc%s' % (site['EASE_row'],site['EASE_column'])][0])
sites['roh_b']=roh_b
sites.to_csv(SM_SITES,index=False)

In [53]:
sites.to_csv(SM_SITES,index=False)

In [52]:
sites

,network,station,lat,lon,s_depth,e_depth,clay,sand,slit,LC2016,LC2017,LC2018,LC2019,EASE_row,EASE_column,roh_b
0,AMMA-CATCH,Banizoumbou,13.53250,2.66040,5,5,0.05,0.90,0.05,40,40,40,40,155,489,1.500000
1,AMMA-CATCH,Belefoungou-Mid,9.79506,1.70994,5,5,0.22,0.49,0.29,126,126,126,126,168,486,1.607042
2,AMMA-CATCH,Belefoungou-Top,9.78986,1.70994,5,5,0.22,0.49,0.29,124,124,124,124,168,486,1.607042
3,AMMA-CATCH,Nalohou-Mid,9.74530,1.60530,5,5,0.22,0.49,0.29,40,40,40,40,168,486,1.607042
4,AMMA-CATCH,Nalohou-Top,9.74407,1.60580,5,5,0.22,0.49,0.29,40,40,40,40,168,486,1.607042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,USCRN,Williams_35_NNW,35.75520,-112.33740,5,5,0.23,0.36,0.41,30,30,30,30,84,181,1.370910
750,USCRN,Wolf_Point_29_ENE,48.30820,-105.10180,5,5,0.23,0.36,0.41,30,30,30,30,51,200,1.286377
751,USCRN,Wolf_Point_34_NE,48.48870,-105.20960,5,5,0.23,0.36,0.41,30,30,30,30,50,200,1.282959
752,USCRN,Yosemite_Village_12_W,37.75920,-119.82080,5,5,0.24,0.49,0.27,121,121,121,121,78,161,1.885292
